In [1]:
!pip install -q --upgrade torch==2.5.1+cu124 torchvision==0.20.1+cu124 torchaudio==2.5.1+cu124 --index-url https://download.pytorch.org/whl/cu124
!pip install -q requests bitsandbytes==0.46.0 transformers==4.48.3 accelerate==1.3.0 openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.2/908.2 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import requests
from IPython.display import Markdown, display, update_display
from google.colab import drive
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig,AutoModelForSpeechSeq2Seq,pipeline
import torch

In [3]:
LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [4]:
drive.mount("/content/drive")
audio_filename = "/content/drive/MyDrive/denver_extract.mp3"
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

Mounted at /content/drive


In [9]:
from transformers import pipeline

pipe = pipeline("automatic-speech-recognition", model="Gwenn-LR/wisper-small-dv")

result = pipe(audio_filename,return_timestamps=True)
transcription = result["text"]

Device set to use cuda:0
/usr/local/lib/python3.12/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


In [10]:
system_message = "You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown."
user_prompt = f"Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.\n{transcription}"

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]


In [11]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [12]:
tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
streamer = TextStreamer(tokenizer)
model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)
outputs = model.generate(inputs, max_new_tokens=2000, streamer=streamer)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown.<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.
 kind of the Confluence Fluance of this whole idea of the Confluence Fluance Week. The merging of two rivers. And as we've kind of seen recently in politics and in the world there's a lot of situations where water is very important right now. And it's a very big issue. So that is the reason the back of the logo is considered water. So let you see the creation of the logo here. So that basically kind of sums up the reason behind the logo and all the 

In [13]:
response = tokenizer.decode(outputs[0])

In [14]:
display(Markdown(response))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown.<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.
 kind of the Confluence Fluance of this whole idea of the Confluence Fluance Week. The merging of two rivers. And as we've kind of seen recently in politics and in the world there's a lot of situations where water is very important right now. And it's a very big issue. So that is the reason the back of the logo is considered water. So let you see the creation of the logo here. So that basically kind of sums up the reason behind the logo and all the meanings behind the symbolism. And you'll hear a little bit more about our Comfluense Week as basically highlighting all of these Indigenous events and things that are happening around Denver so that that we can kind of bring more people together and kind of share this whole idea of indigenous people's day. So thank you. Thank you so much and thanks for your leadership. All right. Welcome to the Denver City Council meeting of Monday, October 9th. Please rise with the pledge of allegiance by Councilman Lopez. I pledge allegiance to the flag of the United States of America. To the republic. I'm going to go back to the standings. One name. Under guy. Indemnisable. The liberty of justice for all. All right. Thank you. Councilman Low-Paz. Madam Secretary, tell your raw call. Black. Clark. Here. Espinosa. Here. Flynn. Gilmour. Here. Here. Cashman. Here. Kenneige. Here. Low-Paz. Here. New? Or Taiga? Here. Susman? Mr. President. Here. 11 in present. 11 members present. We do have a quorum. Approval of the minutes. Are there any corrections to the minutes of October 2nd? Seeing none. Minutes of October 2nd stand to approve. Council unannouncement. So are there any announcements by members of council? Councilman Clark. Thank you, Mr. President. I just wanted to invite everyone down to the first ever Halloween parade on Broadway and Lucky District 7. It will happen on Saturday, October 21st. At 6 o'clock PM, it will move along Broadway from third to Alameda. It's going to be a fun family-friendly event. Everyone's invited to come down, wear a costume. There'll be candy for the kids and there are Tiki, Zambies and 29 Herces and all kinds of fun and funky stuff on the fun and funky part of Broadway. So please join us. October 21st at 6 o'clock for the Broadway Halloween. Thank you Mr. President. All right. Thank you. Councilman Clark. I will be there. All right. Presentations. Madam Secretary, do we have any presentations? None. Mr. President. Communication. Do we have any communications? None. Mr. President. We do have one proclamation in this evening. Proclamation 1127 and observance of the annual indigenous peoples day in the city and county of Denver. Councilman Low-Platest, would you please read it. Thank you Mr. President. We're pride. Proclamation number 17. Well let me just say this differently. Proclamation number 1127 series of 27 team and observance of the second annual indigenous People's Day in the City and County of Denver. Whereas the Council of the City and County of Denver recognizes that the indigenous peoples have lived and flourished on the lands known as the Amerigas and its Taim and Memorial. And that Denver and the surrounding communities are built upon the ancestral homelands of numerous indigenous tribes which include the southern Newt, the Yute Mountain. Newt tribes of Colorado. And whereas the tribal homelands and seasonal encampments of the Arapahoa and Shiaanpipo along the banks of the Cherry Creek and South Platte River, confluence of the events, keep varying to the future settlements that would become the birthplace of the Mile High City. And where as Colorado and encompasses the ancestral homelands of 48 tribes and the city and county of Denver and surrounding communities are home to the descendants of approximately 100 tribal nations. And where as on October 3rd 2016 the city and county of Denver unanimously passed Council Bill 801 series of 2016 officially designating the second Monday of October of each here as indigenous people's day in Denver Colorado. And whereas the council of the city in County of Denver continues to recognize and value the vast contributions made to the community through indigenous people's knowledge, science, philosophy, arts and culture. And through these the city of Denver has developed and thrive. Whereas the indigenous community, especially youth, have made great efforts this year to draw attention to the contributions of indigenous people including Coninfluence Week, drawing record of tenets to a national indigenous youth leadership conference, leading in conversations on inclusion with their peers and supporting increased indigenous youth participation in science and engineering. Now, therefore, be it for claim by this Council of the City and County of Denver. Section one, that the Council of the City and County of Denver celebrates and honors the cultural and foundational contributions of indigenous people to our history. Past, present and future. And continues to promote the education of the Denver community about these historical and contemporary contributions of indigenous people. Section 2. At the City and County of Denver. Col. Rarrow does hereby observe October 9th 2017. As in digits, people's day. Section 3. At the clerk of the City and County of Denver shall attest and affix the seal of the City and County Denver to this proclamation and that a copy be transmit transmit transmit it. Excuse me. To the Denver American Indian Commission. Sydney and County and Denver School District number one. And the Colorado Commission on Indian Affairs. Thank you. Councilman Lopez. Your motion to adopt. Mr. President, I move that proclamation number 1127. Series of 27 team and be adopted. All right. It has been moved in second. It comes with members of council. Councilman Lopez. Thank you Mr. President. It gives me a lot of pleasure and pride to read this proclamation officially for this for the third time. But as indigenous people's day in Denver, we're officially in for the second time. It is always awesome to be able to see. Not just this proclamation come through. Come by my desk, but to see that so many need different people from our community in our council chambers. It was a very beautiful piece of artwork that you presented to us earlier. And it is exactly this spirit that we I'm trapped with this pro-aclimamation. And this actual, the ordinance that created indigenous peoples day when we sat down on a road and as a community we couldn't think of anything else to begin except for the confluence of the two rivers. And those confluence of the two rivers created such a great city. And we live in such an amazing city and we were all proud of it. And sometimes we, and a lot of people from all over the country are out all over the world are proud of it. And sometimes a little too proud of it This is Tom to go back home. I'm kidding when I say that. But the really nice thing about this is that we are celebrating in Disney'sness people's day out of pride for who we are. Who we are as a city and the contributions of Disney'sness people to the city. Not out of spite, not out of a replacement of one culture over the other or out of contempt or disrespect. You know I think of a quote that says so Chavez made very popular and is stuck with me for a very long time and anytime I have the opportunity I speak in front of children and especially children in our community that you know they often second guess themselves and whether they're coming from who they are and And I always say that, you know, it's very important to be proud of who we are from. And the quote that I use from some of the Chai Vizs is, you know, pride in one's own culture does not require a contempt or disrespect of another. Right? And that's very important. It's very important for us to recognize that. No matter who we are or where we come from in this society, your pride in your own culture doesn't require or should not require the contempt contempt or disrespect of another. And man what a year to be for that to just sit on our shoulders for a while for us to think about. And so I wanted to just to thank you all. I think the commission there's going to be a couple of individuals that are coming to come and speak. Thank you for your art. Your lovely artwork for us to see what's in your heart and what now has become. Probably is going to be a very important symbol for the community. And also just for the work. The daily work every single day. We still have a lot of brothers and sisters whose and ancestors once lived in these lands freely. Now, I was standing on the street corners. Right? And poverty without access to services. Right? Without access to sobriety or even housing or jobs. And what a cruel way to pay back a culture that is paved the way for the city to be built upon its shores. So we have a lot of work to do. And these kind of proclamations in these days is not a day off. It's a day on in Denmark. Right? And addressing those critical issues. So I know that my colleagues are very supportive. I'm going to ask you to support this proclamation. I know you always have done in the past. I'm very proud of today. Oh, we made Time Magazine and Newsweek once again. As being a leader in terms of the city's area. Our salary-building indigenous peoples' day. I wanted to make a point on that. Thank you. Councilman Lo-Pez and thank you for sponsoring this. Councilman Maritega. Mr. President I wanna ask that my name be added. I don't think I could add much more to what Councilman Lo-Pez has shared with us. I wanna thank him for bringing this forward and really just appreciate all the contributions that our Native American community has contributed to this great city and great state. I worked in the Lieutenant Governor's Office when the commission on Indian Affairs was created and had the benefit of being able to go down to the four corners for a peace treaty signing ceremony between the youths and the command chiefs that had been sort of at odds with each other for about 100 years. And just being able to participate and that Paawao was pretty awesome. So for those of you who continue to participate in the annual Paawao, it's such a great opportunity for everybody else to enjoy so many of the contributions of the culture. I mean, to see that the dance continues to be carried on as well as the native language from generation to generation is just so incredible because in so many cultures, you know, The people have come here and assimilated to the norms here and they lose their language and lose a lot of the culture. And in the native community, that hasn't happened. That has commitment to just passing that on from generation to generation is so important. And so I'm happy to be a co-sponsor of this tonight. Thank you. All right, thank you. Councilwoman Artega, Councilwoman Kineneige. Thank you very much. And I also want to thank my colleague for bringing this forward. And I just wanted to say a word to the artist about how beautiful and moving I thought this logo was and your description of it. And I think one of the things that is clear is the words sometimes don't convey the power of imagery or music or the other pieces that make up culture. And so I think the art is so important. And when you talked about water, I was also thinking about land. And I guess I just wanted to say thank you. Many of the native of American peoples of Colorado have been at the forefront. Or actually nationally of defending some of the public lians that have been protected over the last few years that are under attack right now. And there are places that you all, the communities have fought to protect but that everyone gets to enjoy. And so I just think that it's an example of where cultural preservation in our sex with environmental protection with you know recreation and all of the other ways that that public lands are so important. And so I think I just want to just say thank you for that because I think we have some very sacred places in our country that are at risk right now. And so as we celebrate I appreciate that there's still a piece of resistance in here. And I think that I just want to mention and a mention of solidarity with that resistance. So thank you. And happy Confluence Week. Thank you. Councilwoman Keneech. And seeing no other comments, I'll just say a couple. And in a time of such divisive ugliness and just despicable behavior from our leadership, the reason I'm so supportive of of indigenous people's days because it means inclusivity. It means respecting all, respecting those who have been silenced on purpose for a long time. And whose history has not been told. And so we celebrate inclusivity in the face of such evil times honestly.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

**Minutes of the Denver City Council Meeting**
**Date:** October 9th, 2023
**Location:** Denver City Council Chambers
**Attendees:**

* Councilman Lopez
* Councilman Low-Paz
* Councilman Clark
* Councilman Gilmour
* Councilman Cashman
* Councilman Kenneige
* Councilwoman Artega
* Councilwoman Keneech
* Councilman Maritega
* Madam Secretary

**Summary:**
The Denver City Council meeting was held on October 9th, 2023, at the Denver City Council Chambers. The meeting began with the pledge of allegiance, followed by the approval of the minutes from the previous meeting. Councilman Clark announced the first-ever Halloween parade in District 7, while Councilman Low-Paz presented a proclamation to observe the second annual Indigenous Peoples' Day in the City and County of Denver.

**Key Discussion Points:**

* The proclamation to observe the second annual Indigenous Peoples' Day in the City and County of Denver was presented by Councilman Low-Paz.
* Councilman Lopez moved to adopt the proclamation, which was seconded by Councilman Maritega.
* Councilman Lopez spoke about the importance of celebrating Indigenous Peoples' Day and the contributions of indigenous people to the city's history and culture.
* Councilwoman Artega and Councilwoman Keneech expressed their support for the proclamation and highlighted the significance of cultural preservation and environmental protection.

**Takeaways:**

* The Denver City Council celebrated the second annual Indigenous Peoples' Day in the City and County of Denver.
* The proclamation recognized the cultural and foundational contributions of indigenous people to the city's history, past, present, and future.
* The city's commitment to promoting education and awareness about the historical and contemporary contributions of indigenous people was reaffirmed.

**Action Items with Owners:**

* Adopt Proclamation 1127, Series of 27, to observe the second annual Indigenous Peoples' Day in the City and County of Denver.
	+ Owner: Councilman Lopez
* Continue to promote education and awareness about the historical and contemporary contributions of indigenous people.
	+ Owner: Councilman Lopez
* Support the preservation of public lands and cultural heritage sites.
	+ Owner: Councilwoman Artega
* Celebrate Confluence Week and highlight the contributions of indigenous people to the city's culture and history.
	+ Owner: Councilwoman Keneech<|eot_id|>